In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from lightgbm import train
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytest import param
import seaborn as sns

import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.fftpack import dct
from scipy.fftpack import idct



 #### -- 모델 준비

In [2]:
# 원전 내부의 충돌체 정보를 네개의 센서 정보만으로 특정해내기
# 데이터 출처 : https://dacon.io/competitions/official/235614/overview/description/
train_features = pd.read_csv('D:/Data/KAERI_dataset/train_features.csv')
train_target = pd.read_csv('D:/Data/KAERI_dataset/train_target.csv')



In [3]:
train_features.shape, train_target.shape



((1050000, 6), (2800, 5))

In [4]:
train_features.head()


,id,Time,S1,S2,S3,S4
0,0,0.000000,0.0,0.0,0.0,0.0
1,0,0.000004,0.0,0.0,0.0,0.0
2,0,0.000008,0.0,0.0,0.0,0.0
3,0,0.000012,0.0,0.0,0.0,0.0
4,0,0.000016,0.0,0.0,0.0,0.0


In [5]:
def find_firt_min_amp(data0,min_amp=937.55):
    data = data0.copy()
    cond_min = (np.abs(data['S1']) > min_amp) | (np.abs(data['S2']) > min_amp) | (np.abs(data['S3']) > min_amp) | (np.abs(data['S4']) > min_amp)
    data_active = data[cond_min]
    data_active = data_active.drop_duplicates(['id'],keep='first')

    return data_active


In [6]:
# scipy에서 Discrete Cosine Transform을 사용, 원하는 만큼만 잘라낼 수 있게 함수 설정
def fourier_trsf(data,sensor,id=10,cutoff=65):
	cond_id = data['id']==id
	wave = data.loc[cond_id,sensor].values
	time = data.loc[cond_id,'Time']
	fft_wave = dct(wave, type=2,n=time.shape[0])
	freq = np.fft.fftfreq(wave.size,d=0.000004)
	cw = np.copy(fft_wave)
	cw[cutoff:]=0
	fft_wave_2 = np.real(idct(cw))
	
	return {"cw":cw[:cutoff],"fft":fft_wave, "freq":freq, "fft_cutoff":fft_wave_2, "time":time, "wave":wave}


In [7]:

def find_unique_freq(data0,head=40):
    data = data0.copy()
    id_list = np.array(data['id'].unique())
    set_dict = {}
    n = data[data['id']==0].shape[0]
    nn = int(n/2)+1

    for s in ['S1','S2','S3','S4']:
        min_set = set(range(0,nn))
        for i in id_list:
            fft_wave = fourier_trsf(data=data,sensor=s,id=i)
            freq = fft_wave['freq'][0:nn]
            amp = fft_wave['fft'][0:nn]
            abs_amp = abs(amp)

            df_wave = pd.DataFrame([freq,amp,abs_amp]).T
            df_wave.columns = ['freq','amp','abs_amp']
            set_i = set(df_wave.sort_values(by='abs_amp',ascending=False).head(head).index)

            min_set = min_set - set_i

        set_dict[s]=min_set
    return set_dict


In [8]:
# 65번째 까지(0~64) 킵해보자
# 일단 feature로 만들어서 넣어주는 함수를 짜자
# column이름은 f1_0~f4_65같은 식으로 넣기
def fourier_feature(data0,cutoff=65):
    data = data0.copy()
    id_list = np.array(data['id'].unique())
    df_id = pd.DataFrame(id_list,columns=['id'])
    df_list = [df_id]

    for s in ['S1','S2','S3','S4']:
        df_s = []
        for i in id_list:
            fft_wave = fourier_trsf(data=data,sensor=s,id=i,cutoff=cutoff)
            amp = fft_wave['cw']
            
            df_wave = pd.DataFrame(amp).T
            df_wave.columns = [s+'_f'+str(n) for n in range(cutoff)]
            df_s.append(df_wave)
        df_sensor = pd.concat(df_s,axis=0).reset_index(drop=True)
        df_list.append(df_sensor)

    df_tot = pd.concat(df_list,axis=1)

    return df_tot


In [9]:
# 확인 해보니, S3가 먼저 신호를 받은 경우가 한번도 없는 것으로 나온다
# 그래서 S3가 항상 고려되지 않은채로 분류 된 것...
# 추후에도 S3값이 고려 될 수 있게, 축을 새로 잡아주기
def reset_axis(data0,new_axis=('A','B','C','D')):
    data = data0.copy()
    
    # A=(S1+S2+S3+S4)/4, B=(S1+S2-S3-S4)/4, C=(S1-S2-S3+S4)/4, D=(S1-S2+S3-S4)/4
    ns1,ns2,ns3,ns4 = data['S1'],data['S2'],data['S3'],data['S4']
    data[new_axis[0]] = (ns1+ns2+ns3+ns4)/4
    data[new_axis[1]] = (ns1+ns2-ns3-ns4)/4
    data[new_axis[2]] = (ns1-ns2-ns3+ns4)/4
    data[new_axis[3]] = (ns1-ns2+ns3-ns4)/4
    data = data.drop(['Time','S1','S2','S3','S4'],axis=1)
    
    return data


In [10]:
# 앞서 수행했던 데이터 전처리 및 feature engineering을 수행해주는 함수
def feature_eng_df(data,cutoff=80):   
    cond_0 = (data['S1'] != 0) | (data['S2'] != 0) | (data['S3'] != 0) | (data['S4'] != 0)
    data_active = data[cond_0]
    data_active = data_active.drop_duplicates(['id'],keep='first')
    
    new_axis = ('A','B','C','D')
    data_new = reset_axis(data,new_axis=new_axis)
    cond_new = (data_new['A'] != 0) | (data_new['B'] != 0) | (data_new['C'] != 0) | (data_new['D'] != 0)  
    data_active_new = data_new[cond_new]
    data_active_new = data_active_new.drop_duplicates(['id'],keep='first')
    
    data_active = data_active.merge(data_active_new,on='id')
    
    for s in ['S1','S2','S3','S4']:
        min_s = data.groupby(by='id').min()[s]
        max_s = data.groupby(by='id').max()[s]
        gap_s = max_s - min_s
        gap_s = gap_s.reset_index()
        gap_s.columns = ['id','gap_'+s]
        data_active = data_active.merge(gap_s,on='id')

    data_active['Time'] = (data_active['Time']*10**6).astype('int')

    data[(data['S2'] != 0)].drop_duplicates(['id'],keep='first')[['id','Time']]

    for s in ['S1','S2','S3','S4']:
        cond_t = (data[s] != 0)
        active_time = data[cond_t].drop_duplicates(['id'],keep='first')[['id','Time']]
        active_time['Time'] = (active_time['Time']*10**6).astype('int')
        active_time.columns = ['id','active_time_'+s]
        data_active = data_active.merge(active_time,on='id')

    data_active['R12'] = (data_active['active_time_S1']+data_active['active_time_S2'])/(data_active['active_time_S3']+data_active['active_time_S4'])
    data_active['R13'] = (data_active['active_time_S1']+data_active['active_time_S3'])/(data_active['active_time_S2']+data_active['active_time_S4'])
    data_active['R14'] = (data_active['active_time_S1']+data_active['active_time_S4'])/(data_active['active_time_S2']+data_active['active_time_S3'])

    data_active['RMS_S'] = (data_active['S1']**2+data_active['S2']**2+data_active['S3']**2+data_active['S4']**2)**0.5
    data_active['RMS_gap'] = (data_active['gap_S1']**2+data_active['gap_S2']**2+data_active['gap_S3']**2+data_active['gap_S4']**2)**0.5
    data_active['RMS_time'] = (data_active['active_time_S1']**2+data_active['active_time_S2']**2+data_active['active_time_S3']**2+data_active['active_time_S4']**2)**0.5
    
    data_fft = fourier_feature(data,cutoff=cutoff)
    data_active = data_active.merge(data_fft,on='id')

    return data_active



In [11]:
df_features = feature_eng_df(train_features)
df_features.head().T



,0,1,2,3,4
id,0.000000e+00,1.000000e+00,2.000000,3.000000e+00,4.000000e+00
Time,4.000000e+01,2.000000e+01,16.000000,3.200000e+01,2.000000e+01
S1,-4.972607e-08,0.000000e+00,-0.000092,0.000000e+00,3.230998e-07
S2,-4.972607e-08,-4.104924e-07,0.000000,-1.783159e-07,0.000000e+00
S3,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
...,...,...,...,...,...
S4_f75,1.684120e+06,-1.518072e+06,-61321.832785,-5.812756e+05,-9.751388e+05
S4_f76,2.884239e+04,-5.615888e+06,-120137.749756,-8.366543e+05,3.916767e+05
S4_f77,3.995064e+05,-1.179631e+06,99635.066101,4.095499e+05,-9.707762e+04
S4_f78,2.543315e+05,2.940736e+06,100054.862204,9.089064e+05,1.146868e+04


In [12]:
targets = list(train_target.columns)[1:]
features = list(df_features.columns)[1:]


In [13]:
df = df_features.merge(train_target,on='id')
df.head()



,id,Time,S1,S2,S3,S4,A,B,C,D,...,S4_f74,S4_f75,S4_f76,S4_f77,S4_f78,S4_f79,X,Y,M,V
0,0,40,-4.972607e-08,-4.972607e-08,0.0,0.0,-2.486304e-08,-2.486304e-08,0.000000e+00,0.000000e+00,...,-6.845378e+05,1.684120e+06,2.884239e+04,3.995064e+05,2.543315e+05,4.671553e+05,0.0,-400.0,50.0,0.4
1,1,20,0.000000e+00,-4.104924e-07,0.0,0.0,-1.026231e-07,-1.026231e-07,1.026231e-07,1.026231e-07,...,-5.314059e+05,-1.518072e+06,-5.615888e+06,-1.179631e+06,2.940736e+06,-5.467193e+06,400.0,0.0,100.0,1.0
2,2,16,-9.210808e-05,0.000000e+00,0.0,0.0,-2.302702e-05,-2.302702e-05,-2.302702e-05,-2.302702e-05,...,4.349310e+04,-6.132183e+04,-1.201377e+05,9.963507e+04,1.000549e+05,2.279359e+05,-300.0,-200.0,25.0,0.4
3,3,32,0.000000e+00,-1.783159e-07,0.0,0.0,-4.457897e-08,-4.457897e-08,4.457897e-08,4.457897e-08,...,-4.198138e+05,-5.812756e+05,-8.366543e+05,4.095499e+05,9.089064e+05,1.626792e+05,200.0,-100.0,150.0,0.4
4,4,20,3.230998e-07,0.000000e+00,0.0,0.0,8.077495e-08,8.077495e-08,8.077495e-08,8.077495e-08,...,-1.710859e+06,-9.751388e+05,3.916767e+05,-9.707762e+04,1.146868e+04,7.479534e+05,-300.0,-100.0,150.0,0.4


In [14]:
# 데이터를 학습용, 검증용으로 분리
df_train, df_val = train_test_split(df[1:],test_size=0.2,train_size=0.8,random_state=2)
df_train.shape, df_val.shape, df.shape



((2239, 348), (560, 348), (2800, 348))

In [15]:
y_train = df_train[targets]
y_train_xy = df_train[['X','Y']]
y_train_mv = df_train[['M','V']]
X_train = df_train[features]

y_val = df_val[targets]
y_val_xy = df_val[['X','Y']]
y_val_mv = df_val[['M','V']]
X_val = df_val[features]



In [16]:
# dacon에서 제공하는 평가 지표 함수. 낮을 수록 좋은 값.
def kaeri_metric(y_true, y_pred):    
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)

def E1(y_true, y_pred):
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)

def E2(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]           
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

In [17]:
train_target.max()
train_target.min()
train_target.mean()

id    1399.5
X        0.0
Y       -5.0
M      100.0
V        0.6
dtype: float64

In [18]:
params = {}
params['n_estimators'] = 100 
params['learning_rate'] = 0.09 # 0.1 # 0.15
params['max_depth'] = 4
params['num_leaves'] = 15 # 5
params['boosting_type'] = 'goss'# 'dart'
params['random_state'] = 2
print('LGBM\n',params)

model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

model.fit(X_train,y_train)

y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)

# 스코어 계산
kaeri_score_train = kaeri_metric(y_train,y_train_pred_xgb)
mae_train = mean_absolute_error(y_train,y_train_pred_xgb)
rmse_train = mean_squared_error(y_train,y_train_pred_xgb)**0.5
kaeri_score_val = kaeri_metric(y_val,y_val_pred_xgb)
mae_val = mean_absolute_error(y_val,y_val_pred_xgb)
rmse_val = mean_squared_error(y_val,y_val_pred_xgb)**0.5

score = pd.DataFrame([round(mae_val/mae_train,2),kaeri_score_train,mae_train, rmse_train, kaeri_score_val,mae_val,rmse_val]).T
score.columns=['mae_ratio(v/t)','kaeri_score_train', 'mae_train', 'rmse_train', 'kaeri_score_val', 'mae_val', 'rmse_val']

score

LGBM
 {'n_estimators': 100, 'learning_rate': 0.09, 'max_depth': 4, 'num_leaves': 15, 'boosting_type': 'goss', 'random_state': 2}


,mae_ratio(v/t),kaeri_score_train,mae_train,rmse_train,kaeri_score_val,mae_val,rmse_val
0,1.56,0.009579,2.592183,4.675246,0.022102,4.055866,8.302376


In [21]:
params = {}
params['n_estimators'] = 1000
params['learning_rate'] = 0.1 # 0.15
params['max_depth'] = 4
params['num_leaves'] = 5
params['boosting_type'] = 'dart'
params['random_state'] = 2
print('LGBM\n',params)

model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

model.fit(X_train,y_train)

y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)

# 스코어 계산
kaeri_score_train = kaeri_metric(y_train,y_train_pred_xgb)
mae_train = mean_absolute_error(y_train,y_train_pred_xgb)
rmse_train = mean_squared_error(y_train,y_train_pred_xgb)**0.5
kaeri_score_val = kaeri_metric(y_val,y_val_pred_xgb)
mae_val = mean_absolute_error(y_val,y_val_pred_xgb)
rmse_val = mean_squared_error(y_val,y_val_pred_xgb)**0.5

score = pd.DataFrame([round(mae_val/mae_train,2),kaeri_score_train,mae_train, rmse_train, kaeri_score_val,mae_val,rmse_val]).T
score.columns=['mae_ratio(v/t)','kaeri_score_train', 'mae_train', 'rmse_train', 'kaeri_score_val', 'mae_val', 'rmse_val']

score

LGBM
 {'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 4, 'num_leaves': 5, 'boosting_type': 'dart', 'random_state': 2}


,mae_ratio(v/t),kaeri_score_train,mae_train,rmse_train,kaeri_score_val,mae_val,rmse_val
0,1.35,0.007085,2.567204,4.669485,0.017345,3.477992,6.642705


In [22]:
params = {}
params['n_estimators'] = 1000
params['learning_rate'] = 0.15 #0.1
params['max_depth'] = 4
params['num_leaves'] = 5
params['boosting_type'] = 'dart'
params['random_state'] = 2
print('LGBM\n',params)

model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

model.fit(X_train,y_train)

y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)

# 스코어 계산
kaeri_score_train = kaeri_metric(y_train,y_train_pred_xgb)
mae_train = mean_absolute_error(y_train,y_train_pred_xgb)
rmse_train = mean_squared_error(y_train,y_train_pred_xgb)**0.5
kaeri_score_val = kaeri_metric(y_val,y_val_pred_xgb)
mae_val = mean_absolute_error(y_val,y_val_pred_xgb)
rmse_val = mean_squared_error(y_val,y_val_pred_xgb)**0.5

score = pd.DataFrame([round(mae_val/mae_train,2),kaeri_score_train,mae_train, rmse_train, kaeri_score_val,mae_val,rmse_val]).T
score.columns=['mae_ratio(v/t)','kaeri_score_train', 'mae_train', 'rmse_train', 'kaeri_score_val', 'mae_val', 'rmse_val']

score

LGBM
 {'n_estimators': 1000, 'learning_rate': 0.15, 'max_depth': 4, 'num_leaves': 5, 'boosting_type': 'dart', 'random_state': 2}


,mae_ratio(v/t),kaeri_score_train,mae_train,rmse_train,kaeri_score_val,mae_val,rmse_val
0,1.53,0.00382,1.794849,3.378536,0.011853,2.74859,5.513833


In [30]:
params = {}
params['n_estimators'] = 1000 #100
params['learning_rate'] = 0.02 #0.15 #0.1
params['max_depth'] = 6
params['num_leaves'] = 5 #5
params['boosting_type'] = 'gbdt'#'dart' #'goss'
params['random_state'] = 2
print('LGBM\n',params)

model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

model.fit(X_train,y_train)

y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)

# 스코어 계산
kaeri_score_train = kaeri_metric(y_train,y_train_pred_xgb)
mae_train = mean_absolute_error(y_train,y_train_pred_xgb)
rmse_train = mean_squared_error(y_train,y_train_pred_xgb)**0.5
kaeri_score_val = kaeri_metric(y_val,y_val_pred_xgb)
mae_val = mean_absolute_error(y_val,y_val_pred_xgb)
rmse_val = mean_squared_error(y_val,y_val_pred_xgb)**0.5

score = pd.DataFrame([round(mae_val/mae_train,2),kaeri_score_train,mae_train, rmse_train, kaeri_score_val,mae_val,rmse_val]).T
score.columns=['mae_ratio(v/t)','kaeri_score_train', 'mae_train', 'rmse_train', 'kaeri_score_val', 'mae_val', 'rmse_val']

score

LGBM
 {'n_estimators': 1000, 'learning_rate': 0.02, 'max_depth': 6, 'num_leaves': 5, 'boosting_type': 'gbdt', 'random_state': 2}


,mae_ratio(v/t),kaeri_score_train,mae_train,rmse_train,kaeri_score_val,mae_val,rmse_val
0,1.45,0.008532,2.551268,4.754366,0.018756,3.708665,7.407671


In [38]:
params = {}
params['n_estimators'] = 1000 #100
params['learning_rate'] = 0.02 #0.15 #0.1
params['max_depth'] = 6
params['num_leaves'] = 6 #5
params['boosting_type'] = 'gbdt'#'dart' #'goss'
params['random_state'] = 2
print('LGBM\n',params)

model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

model.fit(X_train,y_train)

y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)

# 스코어 계산
kaeri_score_train = kaeri_metric(y_train,y_train_pred_xgb)
mae_train = mean_absolute_error(y_train,y_train_pred_xgb)
rmse_train = mean_squared_error(y_train,y_train_pred_xgb)**0.5
kaeri_score_val = kaeri_metric(y_val,y_val_pred_xgb)
mae_val = mean_absolute_error(y_val,y_val_pred_xgb)
rmse_val = mean_squared_error(y_val,y_val_pred_xgb)**0.5

score = pd.DataFrame([round(mae_val/mae_train,2),kaeri_score_train,mae_train, rmse_train, kaeri_score_val,mae_val,rmse_val]).T
score.columns=['mae_ratio(v/t)','kaeri_score_train', 'mae_train', 'rmse_train', 'kaeri_score_val', 'mae_val', 'rmse_val']

score

LGBM
 {'n_estimators': 1000, 'learning_rate': 0.02, 'max_depth': 6, 'num_leaves': 6, 'boosting_type': 'gbdt', 'random_state': 2}


,mae_ratio(v/t),kaeri_score_train,mae_train,rmse_train,kaeri_score_val,mae_val,rmse_val
0,1.64,0.005663,2.060817,3.922484,0.015297,3.375315,7.031755
